In [ ]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind
from google.colab import files # Import files module

def create_hypothesis_test_sheets_auto(input_file, output_file):
    # Load data from specified sheets in the input Excel file
    returns = pd.read_excel(input_file, sheet_name='Returns', header=0)
    rolling_corr = pd.read_excel(input_file, sheet_name='Rolling Correlation', header=0)
    rolling_vol = pd.read_excel(input_file, sheet_name='Rolling Volatility', header=0)

    # Detect rolling correlation columns dynamically (assumed columns containing '_vs_' phrase)
    corr_cols = [col for col in rolling_corr.columns if '_vs_' in col]

    # Use 'with' statement for proper saving and closing
    with pd.ExcelWriter(output_file, engine='openpyxl') as writer:

        # Rolling correlations start producing data at approx week 1, zero-based index 0
        start_index = 1
        end_index = 520 # To include up to the 520th row (index 519)

        for corr_col in corr_cols:
            try:
                # Example: 'Return WHEAT_vs_Return Graphite'
                parts = corr_col.split('_vs_')
                # Remove 'Return ' from each side to get asset names
                asset1 = parts[0].replace('Return ', '').strip()
                asset2 = parts[1].replace('Return ', '').strip()
            except Exception:
                asset1, asset2 = 'Asset1', 'Asset2'

            # Check if volatility regime columns exist for both assets
            vol_col_asset1 = f'Volatility Regime {asset1}'
            vol_col_asset2 = f'Volatility Regime {asset2}'

            if vol_col_asset1 not in rolling_vol.columns or vol_col_asset2 not in rolling_vol.columns:
                print(f"Skipping pair {asset1} vs {asset2} due to missing volatility regime column(s).")
                continue # Skip to the next pair if columns are missing


            # Slice up to the specified end_index
            df = pd.DataFrame()
            # Ensure consistent slicing length
            slice_len = min(len(rolling_corr.loc[start_index:end_index - 1]), len(rolling_vol.loc[start_index:end_index - 1]))

            df['Date'] = rolling_corr.loc[start_index:start_index + slice_len - 1, 'Date'].values
            df['Rolling Correlation (Pair)'] = rolling_corr.loc[start_index:start_index + slice_len - 1, corr_col].values
            df['Volatility Regime Asset 1'] = rolling_vol.loc[start_index:start_index + slice_len - 1, vol_col_asset1].values
            df['Volatility Regime Asset 2'] = rolling_vol.loc[start_index:start_index + slice_len - 1, vol_col_asset2].values

            # Initialize the column for calculated p-value with NaNs
            df['Calculated P-value'] = np.nan

            df['Corr in High Volatility'] = np.where(
                (df['Volatility Regime Asset 1'] == 'High Volatility') & (df['Volatility Regime Asset 2'] == 'High Volatility'),
                df['Rolling Correlation (Pair)'], np.nan
            )
            df['Corr in Low Volatility'] = np.where(
                (df['Volatility Regime Asset 1'] == 'Low Volatility') & (df['Volatility Regime Asset 2'] == 'Low Volatility'),
                df['Rolling Correlation (Pair)'], np.nan
            )

            test_df = df.dropna(subset=['Corr in High Volatility', 'Corr in Low Volatility'], how='all')

            high_corr = test_df['Corr in High Volatility'].dropna()
            low_corr = test_df['Corr in Low Volatility'].dropna()

            p_val = np.nan # Initialize p_val as NaN

            if len(high_corr) > 1 and len(low_corr) > 1:
                t_stat, p_val = ttest_ind(high_corr, low_corr, equal_var=False)


            # Assign the calculated p-value to the second row (index 1) of the new column
            if not np.isnan(p_val):
                 df.loc[1, 'Calculated P-value'] = p_val


            df['Hypothesis Test P-value'] = p_val # This column seems to be for labels, keep it as is for now
            df['Hypothesis Conclusion'] = 'Reject H0' if (not np.isnan(p_val) and p_val < 0.05) else 'Fail to Reject H0'
            df.loc[0, 'Null Hypothesis (Label)'] = 'Mean correlation in high volatility = mean correlation in low volatility'
            df.loc[0, 'Alternative Hypothesis (Label)'] = 'Mean correlation in high volatility ≠ mean correlation in low volatility'

            # Define the desired column order
            output_columns = [
                'Date',
                'Rolling Correlation (Pair)',
                'Volatility Regime Asset 1',
                'Volatility Regime Asset 2',
                'Corr in High Volatility',
                'Corr in Low Volatility',
                'Calculated P-value',  # Insert the new column here
                'Hypothesis Test P-value', # This will still contain the single p_val for row 1
                'Hypothesis Conclusion',
                'Null Hypothesis (Label)',
                'Alternative Hypothesis (Label)'
            ]

            # Reindex the DataFrame to ensure the desired column order
            df = df.reindex(columns=output_columns)


            sheet_name = f'{asset1} vs {asset2}'[:31]
            df.to_excel(writer, sheet_name=sheet_name, index=False)

# Use this exact filename for your input file
# input_file = "Visualization_Returns_All_Currency_Commodity_2015_2024 - Copy.xlsx" # Commented out the hardcoded filename

# Step 1: Upload the Excel file interactively
print("Please upload your Excel file:")
uploaded = files.upload()

# Step 2: Get the filename from the uploaded file
input_file = list(uploaded.keys())[0]
print(f"Uploaded file: {input_file}")

output_file = 'Hypothesis_Test_Results_All_Pairs.xlsx'

create_hypothesis_test_sheets_auto(input_file, output_file)

Please upload your Excel file:


Saving Returns_Correlation_Voloatility_All_Currency_Commodity_2015_2024.xlsx to Returns_Correlation_Voloatility_All_Currency_Commodity_2015_2024 (6).xlsx
Uploaded file: Returns_Correlation_Voloatility_All_Currency_Commodity_2015_2024 (6).xlsx


In [ ]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind
from google.colab import files # Import files module

def create_hypothesis_test_sheets_auto(input_file, output_file):
    # Load data from specified sheets in the input Excel file
    returns = pd.read_excel(input_file, sheet_name='Returns', header=0)
    rolling_corr = pd.read_excel(input_file, sheet_name='Rolling Correlation', header=0)
    rolling_vol = pd.read_excel(input_file, sheet_name='Rolling Volatility', header=0)

    # Detect rolling correlation columns dynamically (assumed columns containing '_vs_' phrase)
    corr_cols = [col for col in rolling_corr.columns if '_vs_' in col]

    # Use 'with' statement for proper saving and closing
    with pd.ExcelWriter(output_file, engine='openpyxl') as writer:

        # Rolling correlations start producing data at approx week 1, zero-based index 0
        start_index = 1
        end_index = 520 # To include up to the 520th row (index 519)

        for corr_col in corr_cols:
            try:
                # Example: 'Return WHEAT_vs_Return Graphite'
                parts = corr_col.split('_vs_')
                # Remove 'Return ' from each side to get asset names
                asset1 = parts[0].replace('Return ', '').strip()
                asset2 = parts[1].replace('Return ', '').strip()
            except Exception:
                asset1, asset2 = 'Asset1', 'Asset2'

            # Check if volatility regime columns exist for both assets
            vol_col_asset1 = f'Volatility Regime {asset1}'
            vol_col_asset2 = f'Volatility Regime {asset2}'

            if vol_col_asset1 not in rolling_vol.columns or vol_col_asset2 not in rolling_vol.columns:
                print(f"Skipping pair {asset1} vs {asset2} due to missing volatility regime column(s).")
                continue # Skip to the next pair if columns are missing


            # Slice up to the specified end_index
            df = pd.DataFrame()
            # Ensure consistent slicing length
            slice_len = min(len(rolling_corr.loc[start_index:end_index - 1]), len(rolling_vol.loc[start_index:end_index - 1]))

            df['Date'] = rolling_corr.loc[start_index:start_index + slice_len - 1, 'Date'].values
            df['Rolling Correlation (Pair)'] = rolling_corr.loc[start_index:start_index + slice_len - 1, corr_col].values
            df['Volatility Regime Asset 1'] = rolling_vol.loc[start_index:start_index + slice_len - 1, vol_col_asset1].values
            df['Volatility Regime Asset 2'] = rolling_vol.loc[start_index:start_index + slice_len - 1, vol_col_asset2].values

            # Initialize the column for calculated p-value with NaNs
            df['Calculated P-value'] = np.nan

            df['Corr in High Volatility'] = np.where(
                (df['Volatility Regime Asset 1'] == 'High Volatility') & (df['Volatility Regime Asset 2'] == 'High Volatility'),
                df['Rolling Correlation (Pair)'], np.nan
            )
            df['Corr in Low Volatility'] = np.where(
                (df['Volatility Regime Asset 1'] == 'Low Volatility') & (df['Volatility Regime Asset 2'] == 'Low Volatility'),
                df['Rolling Correlation (Pair)'], np.nan
            )

            test_df = df.dropna(subset=['Corr in High Volatility', 'Corr in Low Volatility'], how='all')

            high_corr = test_df['Corr in High Volatility'].dropna()
            low_corr = test_df['Corr in Low Volatility'].dropna()

            p_val = np.nan # Initialize p_val as NaN

            if len(high_corr) > 1 and len(low_corr) > 1:
                t_stat, p_val = ttest_ind(high_corr, low_corr, equal_var=False)


            # Assign the calculated p-value to the second row (index 1) of the new column
            if not np.isnan(p_val):
                 df.loc[1, 'Calculated P-value'] = p_val


            df['Hypothesis Test P-value'] = p_val # This column seems to be for labels, keep it as is for now
            df['Hypothesis Conclusion'] = 'Reject H0' if (not np.isnan(p_val) and p_val < 0.05) else 'Fail to Reject H0'
            df.loc[0, 'Null Hypothesis (Label)'] = 'Mean correlation in high volatility = mean correlation in low volatility'
            df.loc[0, 'Alternative Hypothesis (Label)'] = 'Mean correlation in high volatility ≠ mean correlation in low volatility'

            # Define the desired column order
            output_columns = [
                'Date',
                'Rolling Correlation (Pair)',
                'Volatility Regime Asset 1',
                'Volatility Regime Asset 2',
                'Corr in High Volatility',
                'Corr in Low Volatility',
                'Calculated P-value',  # Insert the new column here
                'Hypothesis Test P-value', # This will still contain the single p_val for row 1
                'Hypothesis Conclusion',
                'Null Hypothesis (Label)',
                'Alternative Hypothesis (Label)'
            ]

            # Reindex the DataFrame to ensure the desired column order
            df = df.reindex(columns=output_columns)


            sheet_name = f'{asset1} vs {asset2}'[:31]
            df.to_excel(writer, sheet_name=sheet_name, index=False)

# Use this exact filename for your input file
# input_file = "Visualization_Returns_All_Currency_Commodity_2015_2024 - Copy.xlsx" # Commented out the hardcoded filename

# Step 1: Upload the Excel file interactively
print("Please upload your Excel file:")
uploaded = files.upload()

# Step 2: Get the filename from the uploaded file
input_file = list(uploaded.keys())[0]
print(f"Uploaded file: {input_file}")

output_file = 'Hypothesis_Test_Results_All_Pairs.xlsx'

create_hypothesis_test_sheets_auto(input_file, output_file)

Please upload your Excel file:


Saving Returns_Correlation_Voloatility_All_Currency_Commodity_2015_2024.xlsx to Returns_Correlation_Voloatility_All_Currency_Commodity_2015_2024 (6).xlsx
Uploaded file: Returns_Correlation_Voloatility_All_Currency_Commodity_2015_2024 (6).xlsx


In [ ]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind
from google.colab import files # Import files module

def create_hypothesis_test_sheets_auto(input_file, output_file):
    # Load data from specified sheets in the input Excel file
    returns = pd.read_excel(input_file, sheet_name='Returns', header=0)
    rolling_corr = pd.read_excel(input_file, sheet_name='Rolling Correlation', header=0)
    rolling_vol = pd.read_excel(input_file, sheet_name='Rolling Volatility', header=0)

    # Detect rolling correlation columns dynamically (assumed columns containing '_vs_' phrase)
    corr_cols = [col for col in rolling_corr.columns if '_vs_' in col]

    # Use 'with' statement for proper saving and closing
    with pd.ExcelWriter(output_file, engine='openpyxl') as writer:

        # Rolling correlations start producing data at approx week 1, zero-based index 0
        start_index = 1
        end_index = 520 # To include up to the 520th row (index 519)

        for corr_col in corr_cols:
            try:
                # Example: 'Return WHEAT_vs_Return Graphite'
                parts = corr_col.split('_vs_')
                # Remove 'Return ' from each side to get asset names
                asset1 = parts[0].replace('Return ', '').strip()
                asset2 = parts[1].replace('Return ', '').strip()
            except Exception:
                asset1, asset2 = 'Asset1', 'Asset2'

            # Check if volatility regime columns exist for both assets
            vol_col_asset1 = f'Volatility Regime {asset1}'
            vol_col_asset2 = f'Volatility Regime {asset2}'

            if vol_col_asset1 not in rolling_vol.columns or vol_col_asset2 not in rolling_vol.columns:
                print(f"Skipping pair {asset1} vs {asset2} due to missing volatility regime column(s).")
                continue # Skip to the next pair if columns are missing


            # Slice up to the specified end_index
            df = pd.DataFrame()
            # Ensure consistent slicing length
            slice_len = min(len(rolling_corr.loc[start_index:end_index - 1]), len(rolling_vol.loc[start_index:end_index - 1]))

            df['Date'] = rolling_corr.loc[start_index:start_index + slice_len - 1, 'Date'].values
            df['Rolling Correlation (Pair)'] = rolling_corr.loc[start_index:start_index + slice_len - 1, corr_col].values
            df['Volatility Regime Asset 1'] = rolling_vol.loc[start_index:start_index + slice_len - 1, vol_col_asset1].values
            df['Volatility Regime Asset 2'] = rolling_vol.loc[start_index:start_index + slice_len - 1, vol_col_asset2].values


            df['Corr in High Volatility'] = np.where(
                (df['Volatility Regime Asset 1'] == 'High Volatility') & (df['Volatility Regime Asset 2'] == 'High Volatility'),
                df['Rolling Correlation (Pair)'], np.nan
            )
            df['Corr in Low Volatility'] = np.where(
                (df['Volatility Regime Asset 1'] == 'Low Volatility') & (df['Volatility Regime Asset 2'] == 'Low Volatility'),
                df['Rolling Correlation (Pair)'], np.nan
            )

            test_df = df.dropna(subset=['Corr in High Volatility', 'Corr in Low Volatility'], how='all')

            high_corr = test_df['Corr in High Volatility'].dropna()
            low_corr = test_df['Corr in Low Volatility'].dropna()

            if len(high_corr) > 1 and len(low_corr) > 1:
                t_stat, p_val = ttest_ind(high_corr, low_corr, equal_var=False)
            else:
                p_val = np.nan

            df['Hypothesis Test P-value'] = p_val
            df['Hypothesis Conclusion'] = 'Reject H0' if (not np.isnan(p_val) and p_val < 0.05) else 'Fail to Reject H0'
            df.loc[0, 'Null Hypothesis (Label)'] = 'Mean correlation in high volatility = mean correlation in low volatility'
            df.loc[0, 'Alternative Hypothesis (Label)'] = 'Mean correlation in high volatility ≠ mean correlation in low volatility'

            sheet_name = f'{asset1} vs {asset2}'[:31]
            df.to_excel(writer, sheet_name=sheet_name, index=False)

# Use this exact filename for your input file
# input_file = "Visualization_Returns_All_Currency_Commodity_2015_2024 - Copy.xlsx" # Commented out the hardcoded filename

# Step 1: Upload the Excel file interactively
print("Please upload your Excel file:")
uploaded = files.upload()

# Step 2: Get the filename from the uploaded file
input_file = list(uploaded.keys())[0]
print(f"Uploaded file: {input_file}")

output_file = 'Hypothesis_Test_Results_All_Pairs.xlsx'

create_hypothesis_test_sheets_auto(input_file, output_file)

Please upload your Excel file:


Saving Returns_Correlation_Voloatility_All_Currency_Commodity_2015_2024.xlsx to Returns_Correlation_Voloatility_All_Currency_Commodity_2015_2024 (5).xlsx
Uploaded file: Returns_Correlation_Voloatility_All_Currency_Commodity_2015_2024 (5).xlsx


In [ ]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind
from google.colab import files # Import files module

def create_hypothesis_test_sheets_auto(input_file, output_file):
    # Load data from specified sheets in the input Excel file
    returns = pd.read_excel(input_file, sheet_name='Returns', header=0)
    rolling_corr = pd.read_excel(input_file, sheet_name='Rolling Correlation', header=0)
    rolling_vol = pd.read_excel(input_file, sheet_name='Rolling Volatility', header=0)

    # Detect rolling correlation columns dynamically (assumed columns containing '_vs_' phrase)
    corr_cols = [col for col in rolling_corr.columns if '_vs_' in col]

    # Use 'with' statement for proper saving and closing
    with pd.ExcelWriter(output_file, engine='openpyxl') as writer:

        # Rolling correlations start producing data at approx week 1, zero-based index 0
        start_index = 1
        end_index = 520 # To include up to the 520th row (index 519)

        for corr_col in corr_cols:
            try:
                # Example: 'Return WHEAT_vs_Return Graphite'
                parts = corr_col.split('_vs_')
                # Remove 'Return ' from each side to get asset names
                asset1 = parts[0].replace('Return ', '').strip()
                asset2 = parts[1].replace('Return ', '').strip()
            except Exception:
                asset1, asset2 = 'Asset1', 'Asset2'

            # Check if volatility regime columns exist for both assets
            vol_col_asset1 = f'Volatility Regime {asset1}'
            vol_col_asset2 = f'Volatility Regime {asset2}'

            if vol_col_asset1 not in rolling_vol.columns or vol_col_asset2 not in rolling_vol.columns:
                print(f"Skipping pair {asset1} vs {asset2} due to missing volatility regime column(s).")
                continue # Skip to the next pair if columns are missing


            # Slice up to the specified end_index
            df = pd.DataFrame()
            # Ensure consistent slicing length
            slice_len = min(len(rolling_corr.loc[start_index:end_index - 1]), len(rolling_vol.loc[start_index:end_index - 1]))

            df['Date'] = rolling_corr.loc[start_index:start_index + slice_len - 1, 'Date'].values
            df['Rolling Correlation (Pair)'] = rolling_corr.loc[start_index:start_index + slice_len - 1, corr_col].values
            df['Volatility Regime Asset 1'] = rolling_vol.loc[start_index:start_index + slice_len - 1, vol_col_asset1].values
            df['Volatility Regime Asset 2'] = rolling_vol.loc[start_index:start_index + slice_len - 1, vol_col_asset2].values


            df['Corr in High Volatility'] = np.where(
                (df['Volatility Regime Asset 1'] == 'High Volatility') & (df['Volatility Regime Asset 2'] == 'High Volatility'),
                df['Rolling Correlation (Pair)'], np.nan
            )
            df['Corr in Low Volatility'] = np.where(
                (df['Volatility Regime Asset 1'] == 'Low Volatility') & (df['Volatility Regime Asset 2'] == 'Low Volatility'),
                df['Rolling Correlation (Pair)'], np.nan
            )

            test_df = df.dropna(subset=['Corr in High Volatility', 'Corr in Low Volatility'], how='all')

            high_corr = test_df['Corr in High Volatility'].dropna()
            low_corr = test_df['Corr in Low Volatility'].dropna()

            if len(high_corr) > 1 and len(low_corr) > 1:
                t_stat, p_val = ttest_ind(high_corr, low_corr, equal_var=False)
            else:
                p_val = np.nan

            df['Hypothesis Test P-value'] = p_val
            df['Hypothesis Conclusion'] = 'Reject H0' if (not np.isnan(p_val) and p_val < 0.05) else 'Fail to Reject H0'
            df.loc[0, 'Null Hypothesis (Label)'] = 'Mean correlation in high volatility = mean correlation in low volatility'
            df.loc[0, 'Alternative Hypothesis (Label)'] = 'Mean correlation in high volatility ≠ mean correlation in low volatility'

            sheet_name = f'{asset1} vs {asset2}'[:31]
            df.to_excel(writer, sheet_name=sheet_name, index=False)

# Use this exact filename for your input file
# input_file = "Visualization_Returns_All_Currency_Commodity_2015_2024 - Copy.xlsx" # Commented out the hardcoded filename

# Step 1: Upload the Excel file interactively
print("Please upload your Excel file:")
uploaded = files.upload()

# Step 2: Get the filename from the uploaded file
input_file = list(uploaded.keys())[0]
print(f"Uploaded file: {input_file}")

output_file = 'Hypothesis_Test_Results_All_Pairs.xlsx'

create_hypothesis_test_sheets_auto(input_file, output_file)

Please upload your Excel file:


Saving Returns_Correlation_Voloatility_All_Currency_Commodity_2015_2024.xlsx to Returns_Correlation_Voloatility_All_Currency_Commodity_2015_2024 (5).xlsx
Uploaded file: Returns_Correlation_Voloatility_All_Currency_Commodity_2015_2024 (5).xlsx


In [ ]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind
from google.colab import files # Import files module

def create_hypothesis_test_sheets_auto(input_file, output_file):
    # Load data from specified sheets in the input Excel file
    returns = pd.read_excel(input_file, sheet_name='Returns', header=0)
    rolling_corr = pd.read_excel(input_file, sheet_name='Rolling Correlation', header=0)
    rolling_vol = pd.read_excel(input_file, sheet_name='Rolling Volatility', header=0)

    # Detect rolling correlation columns dynamically (assumed columns containing '_vs_' phrase)
    corr_cols = [col for col in rolling_corr.columns if '_vs_' in col]

    # Use 'with' statement for proper saving and closing
    with pd.ExcelWriter(output_file, engine='openpyxl') as writer:

        # Rolling correlations start producing data at approx week 1, zero-based index 0
        start_index = 1
        end_index = 520 # To include up to the 520th row (index 519)

        for corr_col in corr_cols:
            try:
                # Example: 'Return WHEAT_vs_Return Graphite'
                parts = corr_col.split('_vs_')
                # Remove 'Return ' from each side to get asset names
                asset1 = parts[0].replace('Return ', '').strip()
                asset2 = parts[1].replace('Return ', '').strip()
            except Exception:
                asset1, asset2 = 'Asset1', 'Asset2'

            # Check if volatility regime columns exist for both assets
            vol_col_asset1 = f'Volatility Regime {asset1}'
            vol_col_asset2 = f'Volatility Regime {asset2}'

            if vol_col_asset1 not in rolling_vol.columns or vol_col_asset2 not in rolling_vol.columns:
                print(f"Skipping pair {asset1} vs {asset2} due to missing volatility regime column(s).")
                continue # Skip to the next pair if columns are missing


            # Slice up to the specified end_index
            df = pd.DataFrame()
            # Ensure consistent slicing length
            slice_len = min(len(rolling_corr.loc[start_index:end_index - 1]), len(rolling_vol.loc[start_index:end_index - 1]))

            df['Date'] = rolling_corr.loc[start_index:start_index + slice_len - 1, 'Date'].values
            df['Rolling Correlation (Pair)'] = rolling_corr.loc[start_index:start_index + slice_len - 1, corr_col].values
            df['Volatility Regime Asset 1'] = rolling_vol.loc[start_index:start_index + slice_len - 1, vol_col_asset1].values
            df['Volatility Regime Asset 2'] = rolling_vol.loc[start_index:start_index + slice_len - 1, vol_col_asset2].values


            df['Corr in High Volatility'] = np.where(
                (df['Volatility Regime Asset 1'] == 'High Volatility') & (df['Volatility Regime Asset 2'] == 'High Volatility'),
                df['Rolling Correlation (Pair)'], np.nan
            )
            df['Corr in Low Volatility'] = np.where(
                (df['Volatility Regime Asset 1'] == 'Low Volatility') & (df['Volatility Regime Asset 2'] == 'Low Volatility'),
                df['Rolling Correlation (Pair)'], np.nan
            )

            test_df = df.dropna(subset=['Corr in High Volatility', 'Corr in Low Volatility'], how='all')

            high_corr = test_df['Corr in High Volatility'].dropna()
            low_corr = test_df['Corr in Low Volatility'].dropna()

            if len(high_corr) > 1 and len(low_corr) > 1:
                t_stat, p_val = ttest_ind(high_corr, low_corr, equal_var=False)
            else:
                p_val = np.nan

            df['Hypothesis Test P-value'] = p_val
            df['Hypothesis Conclusion'] = 'Reject H0' if (not np.isnan(p_val) and p_val < 0.05) else 'Fail to Reject H0'
            df.loc[0, 'Null Hypothesis (Label)'] = 'Mean correlation in high volatility = mean correlation in low volatility'
            df.loc[0, 'Alternative Hypothesis (Label)'] = 'Mean correlation in high volatility ≠ mean correlation in low volatility'

            sheet_name = f'{asset1} vs {asset2}'[:31]
            df.to_excel(writer, sheet_name=sheet_name, index=False)

# Use this exact filename for your input file
# input_file = "Visualization_Returns_All_Currency_Commodity_2015_2024 - Copy.xlsx" # Commented out the hardcoded filename

# Step 1: Upload the Excel file interactively
print("Please upload your Excel file:")
uploaded = files.upload()

# Step 2: Get the filename from the uploaded file
input_file = list(uploaded.keys())[0]
print(f"Uploaded file: {input_file}")

output_file = 'Hypothesis_Test_Results_All_Pairs.xlsx'

create_hypothesis_test_sheets_auto(input_file, output_file)

Please upload your Excel file:


Saving Returns_Correlation_Voloatility_All_Currency_Commodity_2015_2024.xlsx to Returns_Correlation_Voloatility_All_Currency_Commodity_2015_2024 (5).xlsx
Uploaded file: Returns_Correlation_Voloatility_All_Currency_Commodity_2015_2024 (5).xlsx
